In [1]:
import pandas as pd

In [3]:
df= pd.read_csv("./OldData_and_newData.csv", usecols=['id', 'article','label'])
df.head()

,id,article,label
0,10909,article 7. - le conseil des professeurs il est...,Not Zone
1,571,article 3. — les ouvrages ou installations son...,Not Zone
2,10509,article l. 365-1. - la responsabilité civile o...,Not Zone
3,2572,article 23. le conseil d'administration ne peu...,Not Zone
4,36,article 14. - sur la base du compte rendu de r...,Not Zone


In [4]:
len(df)

1272

In [5]:
df['Zone'] = df['label'].apply(lambda x: 1 if x=='Zone' else 0)
df

,id,article,label,Zone
0,10909,article 7. - le conseil des professeurs il est...,Not Zone,0
1,571,article 3. — les ouvrages ou installations son...,Not Zone,0
2,10509,article l. 365-1. - la responsabilité civile o...,Not Zone,0
3,2572,article 23. le conseil d'administration ne peu...,Not Zone,0
4,36,article 14. - sur la base du compte rendu de r...,Not Zone,0
...,...,...,...,...
1267,12800,article 4 : les limites des aires centrales so...,Zone,1
1268,4948,article 3. - la directrice de la nature et des...,Zone,1
1269,3303,article 55: les plans d'aménagements du territ...,Not Zone,0
1270,15505,article 78 : mesures de sécurité pour les navi...,Not Zone,0


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['article'], df['Zone'], stratify=df['Zone'])
X_train.head(4)

984     article 2 la ligne de base à partir de laquell...
407     article 25 : 1 - les dispositions législatives...
1063    article 11. les terrains au sud du bassin-cana...
943     article 11. — la laisse de basse mer des îles ...
Name: article, dtype: object

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((954,), (318,), (954,), (318,))

In [8]:
# !pip install tensorflow-text

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\urllib3\response.py", line 437, in _error_catcher
    yield
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\urllib3\response.py", line 560, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\urllib3\response.py", line 526, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Python310\lib\ssl.py", line 1273, in recv_into
    return self.read(nbytes, buffer)



     --                                       0.3/5.0 MB 32.5 kB/s eta 0:02:25



    from_path, content_type = download(link, temp_dir.path)
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_internal\network\download.py", line 147, in __call__
    for chunk in chunks:
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_internal\cli\progress_bars.py", line 53, in _rich_progress_bar
    for chunk in iterable:
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_internal\network\utils.py", line 63, in response_chunks
    for chunk in response.raw.stream(
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\urllib3\response.py", line 621, in stream
    data = self.read(amt=amt, decode_content=decode_content)
  File "C:\Users\BOOR\AppData\Roaming\Python\Python310\site-packages\pip\_vendor\urllib3\response.py", line 559, in read
    with self._error_catcher():
  File "C:\Python310\lib\contextlib.py", line 153, in __exit__
    self.gen.throw(typ, value, traceback)
  File "C:\Us

In [10]:
# !pip install tensorflow-hub

     ------------------------------------ 108.8/108.8 kB 896.1 kB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python310\\Scripts\\make_image_classifier.exe' -> 'C:\\Python310\\Scripts\\make_image_classifier.exe.deleteme'



In [19]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text

ImportError: cannot import name 'keras' from 'tensorflow_text.python' (C:\Users\BOOR\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\__init__.py)

In [4]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [10]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [11]:
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1000, batch_size = 32)

Epoch 1/1000
30/30 [==============================] - 216s 7s/step - loss: 0.6147 - accuracy: 0.6709
Epoch 2/1000
30/30 [==============================] - 215s 7s/step - loss: 0.5932 - accuracy: 0.6908
Epoch 3/1000
30/30 [==============================] - 218s 7s/step - loss: 0.5805 - accuracy: 0.6845
Epoch 4/1000
30/30 [==============================] - 216s 7s/step - loss: 0.5536 - accuracy: 0.7338
Epoch 5/1000
30/30 [==============================] - 213s 7s/step - loss: 0.5336 - accuracy: 0.7400
Epoch 6/1000
30/30 [==============================] - 215s 7s/step - loss: 0.5259 - accuracy: 0.7495
Epoch 7/1000
30/30 [==============================] - 212s 7s/step - loss: 0.5191 - accuracy: 0.7589
Epoch 8/1000
30/30 [==============================] - 218s 7s/step - loss: 0.5032 - accuracy: 0.7736
Epoch 9/1000
30/30 [==============================] - 211s 7s/step - loss: 0.5026 - accuracy: 0.7505
Epoch 10/1000
30/30 [==============================] - 210s 7s/step - loss: 0.4831 - accura

KeyboardInterrupt: 

In [14]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)

10/10 [==============================] - 76s 7s/step
[0.96913    0.67042696 0.77385473 0.0607939  0.5891549  0.17392766
 0.27682638 0.8551876  0.52685523 0.895029   0.94260514 0.8355156
 0.68004596 0.22404698 0.2696047  0.26586172 0.9854815  0.979362
 0.48943228 0.85744965 0.8533508  0.05373267 0.21623142 0.08254911
 0.842073   0.41673496 0.21431936 0.07068339 0.3892468  0.81716347
 0.3884158  0.5314644  0.59008646 0.99309784 0.30643347 0.9891109
 0.3453738  0.784895   0.711525   0.96029663 0.94884616 0.09731998
 0.895029   0.51257366 0.9014123  0.85473    0.39942375 0.7201206
 0.3756973  0.02315357 0.895029   0.2810647  0.8062347  0.05373758
 0.8865283  0.64717025 0.64908284 0.7696797  0.6239556  0.06661753
 0.9048815  0.95078975 0.8745565  0.2605353  0.9051983  0.94593006
 0.07954311 0.2976824  0.1598306  0.24593623 0.28774226 0.8500493
 0.47722217 0.09269641 0.01110126 0.29000732 0.9591153  0.17137542
 0.15717918 0.8914264  0.9850142  0.7840759  0.17593254 0.32966512
 0.9248875  0.0

In [21]:
import pickle as pk
pk.dump(model, open('bert_model.pkl', 'wb'))


INFO:tensorflow:Assets written to: ram://bb837aab-cbb6-489f-8e80-2f01ac8bf100/assets


INFO:tensorflow:Assets written to: ram://bb837aab-cbb6-489f-8e80-2f01ac8bf100/assets


In [23]:
# model = pk.load(open('bert_model.pkl', 'rb'))